In [0]:
import cv2
import numpy as np
from matplotlib import pyplot as plt 

img1 = cv2.imread('tsucuba_left.png',0)  #queryimage # left image
img2 = cv2.imread('tsucuba_right.png',0) #trainimage # right image 
sift = cv2.xfeatures2d.SIFT_create()

 # find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

# FLANN parameters
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)

flann = cv2.FlannBasedMatcher(index_params,search_params)
matches = flann.knnMatch(des1,des2,k=2)

good = []
pts1 = []
pts2 = []

 # ratio test as per Lowe's paper
for i,(m,n) in enumerate(matches):
     if m.distance < 0.75*n.distance:
            good.append(m)
            pts2.append(kp2[m.trainIdx].pt)
            pts1.append(kp1[m.queryIdx].pt)


In [0]:
pts1 = np.int32(pts1)
pts2 = np.int32(pts2)
print (pts1)
F, mask = cv2.findFundamentalMat(pts1,pts2,cv2.FM_RANSAC)
UBIT = 'rohanhem'
 # We select only inlier points
pts1 = pts1[mask.ravel()==1]
pts2 = pts2[mask.ravel()==1]
np.random.seed(sum([ord(c) for c in UBIT]))
np.random.shuffle(pts1)
np.random.shuffle(pts2)  
print (pts1)
pts1 = pts1[:10]
pts2 = pts2[:10]

[[ 30  74]
 [ 31 122]
 [ 35 194]
 [ 35 194]
 [ 35 208]
 [ 36 102]
 [ 36 216]
 [ 37 128]
 [ 38   8]
 [ 38 183]
 [ 39  70]
 [ 40 219]
 [ 42 179]
 [ 42   7]
 [ 42   7]
 [ 43 106]
 [ 43 170]
 [ 44 121]
 [ 44 216]
 [ 44 216]
 [ 45 101]
 [ 45 175]
 [ 45 175]
 [ 46 215]
 [ 47  54]
 [ 48  34]
 [ 48   9]
 [ 48 130]
 [ 49 147]
 [ 50 111]
 [ 50 261]
 [ 50 163]
 [ 51 220]
 [ 52 227]
 [ 54 115]
 [ 54 282]
 [ 54 127]
 [ 54 142]
 [ 55 121]
 [ 56   6]
 [ 56   6]
 [ 57 278]
 [ 57  78]
 [ 57  78]
 [ 57 117]
 [ 57 112]
 [ 58 150]
 [ 58 130]
 [ 58 153]
 [ 58 136]
 [ 63   9]
 [ 63   9]
 [ 64  64]
 [ 64  64]
 [ 66 153]
 [ 67 213]
 [ 69 149]
 [ 70  12]
 [ 70 222]
 [ 70 222]
 [ 72 141]
 [ 73  39]
 [ 74 141]
 [ 75 151]
 [ 75 194]
 [ 76 183]
 [ 77 147]
 [ 82 176]
 [ 82 142]
 [ 84 114]
 [ 86 153]
 [ 89 140]
 [ 90 118]
 [ 91 179]
 [ 91 144]
 [ 92 166]
 [ 93 112]
 [ 93 112]
 [ 95 149]
 [ 96 113]
 [ 97 151]
 [ 99 283]
 [101 144]
 [101  78]
 [101  78]
 [101  43]
 [102 271]
 [103 189]
 [103 189]
 [103 151]
 [103  73]

In [0]:
def drawlines(img1,img2,lines,pts1,pts2):
    r,c = img1.shape
    img1 = cv2.cvtColor(img1,cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2,cv2.COLOR_GRAY2BGR)
    for r,pt1,pt2 in zip(lines,pts1,pts2):
        color = tuple(np.random.randint(0,255,3).tolist())
        x0,y0 = map(int, [0, -r[2]/r[1] ])
        x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
        img1 = cv2.line(img1, (x0,y0), (x1,y1), color,1)
        img1 = cv2.circle(img1,tuple(pt1),5,color,-1)
        img2 = cv2.circle(img2,tuple(pt2),5,color,-1)
    return img1,img2

In [0]:
# Find epilines corresponding to points in right image (second image) and
# drawing its lines on left image
lines1 = cv2.computeCorrespondEpilines(pts2.reshape(-1,1,2), 2,F)
lines1 = lines1.reshape(-1,3)
img5,img6 = drawlines(img1,img2,lines1,pts1,pts2)

 # Find epilines corresponding to points in left image (first image) and
# drawing its lines on right image
lines2 = cv2.computeCorrespondEpilines(pts1.reshape(-1,1,2), 1,F)
lines2 = lines2.reshape(-1,3)
img3,img4 = drawlines(img2,img1,lines2,pts2,pts1)

plt.subplot(121),plt.imshow(img5)
plt.subplot(122),plt.imshow(img3)
plt.show()
cv2.imwrite('task2_epi_right.jpg',img3)
cv2.imwrite('task2_epi_left.jpg',img5)